# LangChain QA Panel App

This notebook shows how to make this app:

In [7]:
#!pip install langchain openai chromadb tiktoken pypdf panel

In [2]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from pdf_handler import pdf_convert, pdf_classifier
from dotenv import load_dotenv
import panel as pn
import tempfile


In [3]:
load_dotenv()

pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="630px",
    header_background="#F08080",
)

In [ ]:
# file_input = pn.widgets.FileInput(width=290, margin=(5,0,5,0))
file_input = pn.widgets.FileInput(width=630, margin=(5,0,5,0))

# openaikey = pn.widgets.PasswordInput(
#     value="", placeholder="Enter your OpenAI API Key here...", width=325, height=48, margin=(5,10,5,15)
# )
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=100, toolbar=False, margin=(5,10,5,0)
)
run_button = pn.widgets.Button(name="Run!", margin=(5,10,5,0))

# select_k = pn.widgets.IntSlider(
#     name="Number of relevant chunks", start=1, end=5, step=1, value=1, margin=(5,10,5,15)
# )
# select_chain_type = pn.widgets.RadioButtonGroup(
#     name='Chain type', 
#     options=['stuff', 'map_reduce', "refine", "map_rerank"]
# )

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=0, width=640),
    # pn.Column(prompt, run_button, margin=0, width=300),
    # pn.Card(
    #     "Chain type:",
    #     pn.Column(select_chain_type, select_k, margin=(10,10,10,-5)),
    #     title="Advanced settings", margin=5, width=325
    # ), 
    width=660
)

In [4]:
def get_db(file):
    # load document
    if pdf_classifier(file) == 0:
        pdf_convert()
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    return db

def qa(query, chain_type, k, db):
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [1]:
# result = qa("./.cache/temp.pdf", "what is hypertension?", "stuff", 1)
# print(result)

In [2]:
convos = []  # store all panel objects in a list
db = None
def qa_result(_):
    # if openaikey.value != "":
    #     os.environ["OPENAI_API_KEY"] = openaikey.value
    
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("./.cache/temp.pdf")
    
        prompt_text = prompt.value
        if prompt_text:
            try:
                select_chain_type = 'stuff' # select_chain_type.value
                select_k = 1 # select_k.value
                global db
                if not db:
                    db = get_db("./.cache/temp.pdf")
                result= qa(query=prompt_text, chain_type=select_chain_type, k=select_k, db=db)
            except Exception as err:
                db = None
                # result = {"result": f"Error: {err}"}
                result = {"result": f"Sorry! uploaded file is not compatible, Please try another document."}
            content = [
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        # "Relevant source text:",
                        # pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ]
            convos.extend(content)
            #return convos
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [ ]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [ ]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=630, height=500, scroll=True)

In [ ]:
# layout
pn.Column(
    pn.pane.Markdown(
    # """
    # ## \U0001F60A! Question Answering with your PDF file
    
    # 1) Upload a PDF. 2) Enter OpenAI API key. This costs $. Set up billing at [OpenAI](https://platform.openai.com/account). 3) Type a question and click "Run".
    
    # """),
    """
    ## \U0001F60A! Question Answering with your PDF file
    
    """),
    # pn.Row(file_input, openaikey),
    pn.Row(file_input),
    widgets,
    output,
).servable()